In [ ]:
import mlflow.keras
import mlflow
import mlflow.tensorflow
from table_functions import *
from analyze_visualisation import *
from ml_models.decision_tree import *
from ml_models.random_forest import *
from ml_models.xg_boost import *
from ml_models.neural_network_train import *

In [ ]:
# TO MUSI BYC USTAWIONE ZAWSZE TAK SAMO, a potem jak sie chce umieszczac dane modele w innym folderze to zmienia sie nazwe  artifact_directory = "other_classification"
# jak chce sie wlaczyc potem mlflow w przegladarce, trzeba w terminalu przejsc do lokalizacji 'src' i wpisac komende mlflow ui
ml_runs_directory = r"C:\Users\dlxpmx8\Desktop\Projekt_AI\meb_process_data_analysis\src\mlruns"
mlflow.set_tracking_uri(f"file://{ml_runs_directory}")

In [ ]:
data_keys = ['x_train', 'x_valid', 'x_test', 'y_train', 'y_valid', 'y_test']
ml_data = {key: None for key in data_keys}
for key in ml_data:
    ml_data[key] = read_csv(f'{key}.csv')

<h4>Drzewo decyzyjne</h4>

In [ ]:
clf = decision_tree_model(*ml_data.values(), run_name_="third_test_run")
#print_stats(clf, ml_data['x_test'], ml_data['y_test'])

<h4>Sieci neuronowe<h4>

In [ ]:
compile_fit_evaluate_model(*ml_data.values(), epochs_=5, batch_size_=64, comment="standarized data", run_name_="test_run_5_epochs")